In [36]:
import numpy as np
import pandas as pd
from qiskit.quantum_info import SparsePauliOp
from susy_qm import calculate_Hamiltonian

In [2]:
potential = 'DW'

In [42]:
cutoffs = [16]
for cutoff in cutoffs:
    print(cutoff, potential)
    H = calculate_Hamiltonian(cutoff, potential)
    hamiltonian = SparsePauliOp.from_operator(H)
    num_qubits = hamiltonian.num_qubits
        
    eigenvalues, eigenvectors = np.linalg.eig(H)
    min_eigenvalue = min(eigenvalues.real)
    min_index = np.argmin(eigenvalues)
    min_eigenvalue = eigenvalues[min_index]
    min_eigenvector = np.asarray(eigenvectors[:, min_index])

    num_levels = 1
    min_ev = eigenvalues.argsort()[:num_levels]
    for e_lev in range(num_levels):
        eig_vec = np.asarray(eigenvectors[:, min_ev[e_lev]].real)
        eig_vec[np.abs(eig_vec) < 1e-9] = 0
        eig_vec = eig_vec.reshape((2, cutoff))
        print(eig_vec)
        print("#####################################")

16 DW
[[ 7.90444362e-01 -5.83054050e-01  1.40808200e-01  8.43368377e-02
  -8.41227336e-02  2.15341878e-02  1.58155893e-02 -1.95095017e-02
   5.61182898e-03  6.36916635e-03 -6.49531964e-03  1.36568657e-04
   3.38516909e-03 -1.86669411e-03 -1.19433082e-03  2.07081883e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
#####################################


In [43]:
def highlight_nonzero(val):
    return 'background-color: green' if val != 0 else ''

df = pd.DataFrame(H.real)
df = df.style.map(highlight_nonzero).format(precision=1)
df


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,2.4,2.5,1.8,0.9,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.5,5.9,5.0,4.3,1.7,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.8,5.0,10.9,8.0,7.8,2.7,2.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.9,4.3,8.0,17.4,11.3,12.3,3.9,3.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.6,1.7,7.8,11.3,25.4,15.0,17.8,5.1,5.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.4,2.7,12.3,15.0,34.9,19.1,24.3,6.5,6.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,2.4,3.9,17.8,19.1,45.9,23.4,31.8,7.9,8.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,3.6,5.1,24.3,23.4,58.4,28.0,40.3,9.5,11.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,5.1,6.5,31.8,28.0,72.4,32.9,49.8,11.1,13.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,6.9,7.9,40.3,32.9,87.9,38.0,60.3,12.8,16.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
matrix = H.real
evec = min_eigenvector

# Threshold to consider "non-zero" contributions
threshold = 1e-4
active_indices = [i for i, val in enumerate(evec) if abs(val) > threshold]

# Convert matrix to DataFrame for styling
df = pd.DataFrame(matrix)

# Define cell highlighting function
def highlight_active_contribution(val, row_idx, col_idx):
    if row_idx in active_indices and col_idx in active_indices:
        return 'background-color: green'
    return ''

# Apply styling
styled_df = df.style.apply(
    lambda data: np.vectorize(highlight_active_contribution)(data, *np.indices(data.shape)),
    axis=None
).format("{:.1f}")

styled_df


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,2.4,2.5,1.8,0.9,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.5,5.9,5.0,4.3,1.7,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.8,5.0,10.9,8.0,7.8,2.7,2.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.9,4.3,8.0,17.4,11.3,12.3,3.9,3.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.6,1.7,7.8,11.3,25.4,15.0,17.8,5.1,5.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.4,2.7,12.3,15.0,34.9,19.1,24.3,6.5,6.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,2.4,3.9,17.8,19.1,45.9,23.4,31.8,7.9,8.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,3.6,5.1,24.3,23.4,58.4,28.0,40.3,9.5,11.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,5.1,6.5,31.8,28.0,72.4,32.9,49.8,11.1,13.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,6.9,7.9,40.3,32.9,87.9,38.0,60.3,12.8,16.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
def binary_state(index, num_qubits=num_qubits):
    return format(index, f'0{num_qubits}b')

states = [binary_state(i) for i in active_indices]
print("Active basis states:")
for s in states:
    print(s)

# Transpose to get per-qubit participation
bit_matrix = np.array([[int(bit) for bit in state] for state in states])
qubit_activity = bit_matrix.sum(axis=0)

print("\nQubit activity (number of '1's per qubit):")
for i, count in enumerate(qubit_activity[::-1]):  # reverse to match q2, q1, q0
    print(f"Qubit {i}: {'active' if count > 0 else 'inactive'} ({count} times)")


Active basis states:
00000
00001
00010
00011
00100
00101
00110
00111
01000
01001
01010
01011
01100
01101
01110
01111

Qubit activity (number of '1's per qubit):
Qubit 0: active (8 times)
Qubit 1: active (8 times)
Qubit 2: active (8 times)
Qubit 3: active (8 times)
Qubit 4: inactive (0 times)


In [63]:
import pennylane as qml
import numpy as np

# Use default.qubit to get full statevector
dev = qml.device("default.qubit", wires=5, shots=1024)

@qml.qnode(dev)
def circuit(params):
    # Apply gates according to your ansatz (reversed wire-to-qubit mapping)

    # q0 (wire 4)
    qml.RY(params[0], wires=4)

    # q1 (wire 3)
    qml.RY(params[1], wires=3)

    # Controlled-RY from q0 (wire 4) to q1 (wire 3)
    qml.ctrl(qml.RY, control=4)(params[2], wires=3)

    # q2 (wire 2)
    qml.RY(params[3], wires=2)

    # q3 (wire 1) — no gate
    # q4 (wire 0) — no gate

    return qml.probs()

In [64]:
# Set example parameters
params = np.array([0.5, 0.3, 0.7, 0.2])
# Run the circuit
state = circuit(params)


In [65]:
state

array([0.92285156, 0.03125   , 0.0234375 , 0.01269531, 0.00976562,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

In [66]:
# Print amplitude of |01000⟩ → binary index 8
target_index = 2
amplitude = state[target_index]

print(f"Amplitude of |01000⟩: {amplitude}")
#print(f"Probability of |01000⟩: {abs(amplitude)**2:.6f}")


Amplitude of |01000⟩: 0.0234375
